## Automate download of the zipped county files

In [25]:
# for the webpage (per state) automate the download of the ZIP county files as outlined below

In [26]:
# Option 0
###
# parsing HTML code (downloaded from  USDA webapp pereodically - once a month? to check for updates there and download-unzip-ingest new versions of the files:
# this is the main URL where web app resides - it generates the table with county lists and timestamps when updated
# https://websoilsurvey.sc.egov.usda.gov/App/WebSoilSurvey.aspx 
# the source file download to /data_in/html/

# Lesha - TO DO:
'''
1. Manually - Download .html for MO, MS, TX, LA (AR is done) - DONE
2. From Option 0 routine make a function to go over each state .html, extract list of .zip and export as CSV
3. Append all CSV together
4. Make a log file with STATE | FIPS | Version date | Date processed
5. Function to download files for each link (checks log if zip file already exists/processed or link is newer)

Note: I changed ABSOLUTE links to "Trubin/..." to the relative links to the folders in this REPO - use it as example
      for future work.
'''


'\n1. Manually - Download .html for MO, MS, TX, LA (AR is done) - DONE\n2. From Option 0 routine make a function to go over each state .html, extract list of .zip and export as CSV\n3. Append all CSV together\n4. Make a log file with STATE | FIPS | Version date | Date processed\n5. Function to download files for each link (checks log if zip file already exists/processed or link is newer)\n\nNote: I changed ABSOLUTE links to "Trubin/..." to the relative links to the folders in this REPO - use it as example\n      for future work.\n'

In [27]:
# pip install lxml - that was needed for pd.read_html() - solved without it so far

In [51]:
import pandas as pd
import os 
from string import ascii_letters
import numpy as np
import glob

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [29]:
# the whole html of 12K+ rows apparently only has 20 AK (Alaska) counties - so apparently the HTML need to be manually
# pulled per state at least - that seems to work

In [40]:
#all HTML files in directory
os.chdir("../html")
for html_files in glob.glob("*.html"):
    print(html_files)

20220715_view-source_https___websoilsurvey.sc.egov.usda.gov_App_WebSoilSurvey.aspx.html
20220722-AK-view-source_https___websoilsurvey.sc.egov.usda.gov_App_WebSoilSurvey.aspx.html
20220722-AR.html
20220726-LA.html
20220726-MO.html
20220726-MS.html
20220726-TX.html


In [46]:
html_files = ["*.html"]
files = []
for type in html_files:
     this_type_files = glob.glob(type)
     files += this_type_files

print(files)

['20220715_view-source_https___websoilsurvey.sc.egov.usda.gov_App_WebSoilSurvey.aspx.html', '20220722-AK-view-source_https___websoilsurvey.sc.egov.usda.gov_App_WebSoilSurvey.aspx.html', '20220722-AR.html', '20220726-LA.html', '20220726-MO.html', '20220726-MS.html', '20220726-TX.html']


In [48]:
#all_rec = glob.glob('D:\python\*.html')
##print(all_rec)
#list_data = []
#
#for filename in all_rec:
#    data = pd.read_html(filename)
#    list_data.append(data)
#
#list_data # showing array of all html data  
#
##remove this line
#list_data =list_data[0] 
##remove this line
#list_data =list_data[-1] 
#
#pd.DataFrame(list_data).reset_index(drop=True) #replace concat with DataFrame

In [52]:
root_dir = '../html'

for directory, subdirectories, files in os.walk(root_dir):
    for file in files:
        print os.path.join(directory, file)

SyntaxError: invalid syntax (Temp/ipykernel_1080/1126265189.py, line 5)

In [ ]:
'''
Make function out of below and apply to folder with HTML files
'''

#Make a list from big HTML file
#list = [ line for line in open(r'../data_in/html/20220715_view-source_https___websoilsurvey.sc.egov.usda.gov_App_WebSoilSurvey.aspx.html') if '.zip' in line]
# testing per state HTML - AK
# list = [ line for line in open(r'../data_in/html/20220722-AK-view-source_https___websoilsurvey.sc.egov.usda.gov_App_WebSoilSurvey.aspx.html') if '.zip' in line]

# testing per state HTML - AR
list = [ line for line in open(r'../data_in/html/20220722-AR.html') if '.zip' in line]
#List to dataframe
df = pd.DataFrame(list)
#name the column
df.columns = ['1']
#data cleaning
df['1'].str.split('                                                          ', expand=True)
df = df[df['1'].str.contains('TemplateDB') == False]
df['1'] = df['1'].map(lambda x: x.lstrip('</td></tr><tr><td class="line-number" value="'))
df[['1', '2']] = df['1'].str.split('                                                      <span class="html-tag">&lt;a <span class="html-attribute-name">href</span>="<a class="html-attribute-value html-external-link" target="_blank" href="', expand=True)
df = df.drop('1', axis=1)
df = df.replace(to_replace='None', value=np.nan).dropna()
df['1'] = df['2'].str.split('" rel=').str[0]
df['2'] = df['1'].str.split('" rel=').str[0]
df = df.drop('1', axis=1)
#result export to CSV
df.to_csv('../data_out/AR_links_list.csv', index=False)  

# when making function, maybe df append of all records? - see example in cell below from SmartRice project


In [ ]:
# functions to clean and append files per consultant

def to_df_csv_path(path):
    '''
    accepts path to folder with CSVs all containing the same structured tables for a single consultant (Consultant ID as a string)
    1. define a list of CSV files from the path
    for each file in the list:
    1. load csv as df
    2. drop empty columns
    3. for string columns - strip beginning/end spaces
    4. append all together.-> return df
    '''
    csvlst = os.listdir(path)
    print(csvlst)
    os.chdir(path)
    
    df1 = pd.DataFrame()
    for file in range(len(csvlst)):
        if df1.empty:
            df1 = pd.read_csv(csvlst[file])
            file+=1
        else:
            df2 = pd.read_csv(csvlst[file])
            df1 = df1.append(df2, ignore_index=True, sort = True) # sort = True is added after received warning about further versions of Anaconda/Pandas
            file+=1
    
      
    # cleaning '---' records (this is specific for this data set, can be generalized later
    df1.replace({'---': None}, inplace=True)
    
    # dropping empty columns
    df1.dropna(axis=1, how='all', inplace=True)
    
    # counting and then dropping completely duplicated records - sometimes consultants report previous year with the records of current year, 
    #thus providing complete duplicate for some records
    print('There were completely duplicated rows before cleanup: ', df1.duplicated().sum())
    #dfdup = df1.loc[df1.duplicated(keep=False), :]
    df1.drop_duplicates(keep='first', inplace=True)
    
    return df1

In [ ]:
df.info()
# 137 counties for AK
# 70 counties for AR

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70 entries, 0 to 138
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   2       70 non-null     object
dtypes: object(1)
memory usage: 1.1+ KB


In [ ]:
#export to csv
#os.makedirs('../soils-ssurgo', exist_ok=True)
df.to_csv('../data_out/AR_links_list.csv', index=False)  

Other options

In [ ]:
#Option 1
# Read zip files from page, download file, extract and stream output
from io import BytesIO
from zipfile import ZipFile
import urllib.request
import os,sys,requests,csv
from bs4 import BeautifulSoup

# check for download directory existence; create if not there
if not os.path.isdir('C:\\Users\\trubin\\Downloads'):
    os.makedirs('C:\\Users\\trubin\\Downloads')

# Get labels and zip file download links
mainurl = "https://websoilsurvey.sc.egov.usda.gov" # ??
#

# below is example of cached URL for zipfile with MS Access template included - we do not need it
# https://websoilsurvey.sc.egov.usda.gov/DSD/Download/Cache/SSA/wss_SSA_AL001_soildb_US_2003_[2021-09-15].zip

url = "http://websoilsurvey.sc.egov.usda.gov/DSD/Download/{CACHENAME}/{SUBFOLDER}/{FILENAME}"

# get page and setup BeautifulSoup
r = requests.get(url)
soup = BeautifulSoup(r.content, "html.parser")


# Get all file labels and filter so only use CSVs
mainlabel = soup.find_all("td", {"class": "labelOptional_ind"})
for td in mainlabel:
    if "_AL" in td.text:
        print(td.text)

###

# Get all <a href> urls
for link in soup.find_all('a'):
    print(mainurl + link.get('href'))

# QUESTION: HOW CAN I LOOP THROUGH ALL FILE LABELS AND FIND ONLY THE
# CSV LABELS AND THEIR CORRESPONDING ZIP DOWNLOAD LINK, SKIPPING ANY
# XML LABELS/LINKS, THEN LOOP AND EXECUTE THE CODE BELOW FOR EACH, 
# REPLACING zipfilename WITH THE MAIN LABEL AND zipurl WITH THE ZIP 
# DOWNLOAD LINK?

# Test downloading and streaming
zipfilename = "wss_SSA_AL019_[2021-09-15].zip"
zipurl = "http://websoilsurvey.sc.egov.usda.gov/DSD/Download/{CACHENAME}/{SUBFOLDER}/{FILENAME}"
outputFilename = "C:\\Users\\trubin\\Downloads" + zipfilename

# Unzip and stream CSV file
url = urllib.request.urlopen(zipurl)
zippedData = url.read()

# Save zip file to disk
print ("Saving to ",outputFilename)
output = open(outputFilename,'wb')
output.write(zippedData)
output.close()

# Unzip and stream CSV file
with ZipFile(BytesIO(zippedData)) as my_zip_file:
   for contained_file in my_zip_file.namelist():
    with open(("unzipped_and_read_" + contained_file + ".file"), "wb") as output:
        for line in my_zip_file.open(contained_file).readlines():
            print(line)


In [ ]:
#Option 2
import urllib.request

user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'

url = "http://websoilsurvey.sc.egov.usda.gov"
headers={'User-Agent':user_agent,} 

request=urllib.request.Request(url,None,headers) #The assembled request
response = urllib.request.urlopen(request)
data = response.read() # The data u need

In [ ]:
#Option 3
opener = urllib.request.build_opener()
opener.addheaders = [('User-Agent', 'MyApp/1.0')]
urllib.request.install_opener(opener)
urllib.request.urlretrieve(
  "http://websoilsurvey.sc.egov.usda.gov/DSD/Download/{CACHENAME}/{SUBFOLDER}",
   "wss_SSA_AL019_[2021-09-15].zip")
outputFilename = "C:\\Users\\trubin\\Downloads" + zipfilename
url = urllib.request.urlopen(zipurl)
zippedData = url.read()
# Save zip file to disk
print ("Saving to ",outputFilename)
output = open(outputFilename,'wb')
output.write(zippedData)
output.close()

In [ ]:
# create txt log of zipfiles downloaded


In [ ]:
# do not download the file if already in the log

In [ ]:
# if not in the log or the file on website is newer

# 1. download
# 2. update log
# 3. unzip to SSURGO folder
# 4. delete the downloaded zipped file